In [1]:
!pip install pymongo[srv]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.5 MB/s eta 0:00:00


In [2]:
!pip install python-dotenv

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
from dotenv import load_dotenv
load_dotenv('api.env')
mongo_access=os.environ.get("mongodb_access")
# Create a new client and connect to the server
client = MongoClient(mongo_access, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

In [ ]:
db=client["Steam_Game"]
collection=db["Steam_Embedding"]
collection.find()

In [5]:
from sentence_transformers import SentenceTransformer, util
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
def get_embedding(text):
    if not isinstance(text, str) or not text.strip():
        print("Skipping invalid text")
        return []
    return embedding_model.encode(text).tolist()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
import re
def extract_metadata_tags(text):
    tags = {
        "year": None,
        "price": None,
        "review": None
    }

    year_match = re.search(r"\b(19|20)\d{2}\b", text)
    if year_match:
        tags["year"] = int(year_match.group())

    price_match = re.search(r"\$?(\d+)(\.\d{1,2})?", text)
    if price_match:
        tags["price"] = float(price_match.group(1))

    for sentiment in ["Positive", "Mixed", "Negative"]:
        if sentiment.lower() in text.lower():
            tags["review"] = sentiment
            break

    return tags


In [7]:
def vector_search_description(user_query, collection, limit=100):
    """
    Perform a vector search and then filter results using extracted metadata.
    """

    # Step 1: Extract metadata tags from user query
    tags = extract_metadata_tags(user_query)

    # Step 2: Generate embedding for the query
    query_embedding = get_embedding(user_query)
    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Step 3: Run vector search
    pipeline = [
        {
            "$vectorSearch": {
                "index": "des_embed",
                "queryVector": query_embedding,
                "path": "embedding_description",
                "numCandidates": 400,
                "limit": limit,
            }
        },
        {"$unset": "embedding"},
        {
            "$project": {
                "_id": 0,
                "name": 1,
                "description": 1,
                "all_reviews": 1,
                "release_date": 1,
                "developer": 1,
                "publisher": 1,
                "price": 1,
                "score": {"$meta": "vectorSearchScore"},
            }
        }
    ]

    results = list(collection.aggregate(pipeline))

    filtered_results = []
    for game in results:
        # Check year
        if tags["year"] is not None:
          if not game.get("release_date") or game["release_date"].year != tags["year"]:
            continue


        if tags["price"] is not None:
          if not game.get("price"):
            continue
          try:
            game_price = float(str(game["price"]).replace("$", "").strip())
            if game_price > tags["price"]:
              continue
          except ValueError:
            continue  # Skip games with malformed price


        # Check review
        if tags["review"]:
            if not game.get("all_reviews") or tags["review"].lower() not in game["all_reviews"].lower():
                continue

        # If all filters pass
        filtered_results.append(game)

    return filtered_results[:5] if filtered_results else results[:5]  # fallback to original results if none match filter

In [8]:
def vector_search_name(user_query, collection, limit=1):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    vector_search_stage = {
        "$vectorSearch": {
            "index": "name_embed",
            "queryVector": query_embedding,
            "path": "embedding_name",
            "numCandidates": 400,
            "limit": limit,
        }
    }

    unset_stage = {
        "$unset": "embedding"
    }

    project_stage = {
            "$project": {
                "_id": 0,
                "name": 1,  # Game name
                "description": 1,  # Game description
                "all_reviews": 1, #Game review
                "release_date": 1,  # Release date
                "developer": 1,  # Developer name
                "publisher": 1,  # Publisher name
                "price": 1,  # Game price
                "score": {
                    "$meta": "vectorSearchScore"
                }
            }
        }

    pipeline = [vector_search_stage, unset_stage, project_stage]

    # Execute the search
    results = collection.aggregate(pipeline)

    return list(results)

In [9]:
user_query = "Recommend farming games in 2023 with positive reviews"
games_des = vector_search_description(user_query, collection)

for game in games_des:
    print(game)

{'name': 'Peaceful Days', 'all_reviews': 'Very Positive', 'release_date': datetime.datetime(2023, 10, 16, 0, 0), 'developer': 'Nhat Nguyen', 'publisher': 'Nhat Nguyen', 'price': '$0.80', 'description': 'peaceful days farming rpg heavily inspired harvest moon series game build dream scratch raise animals interact town folks take part interesting festivals discover many secrets', 'score': 0.7773895859718323}
{'name': 'Lawnmower Game 3: Horror', 'all_reviews': 'Mostly Positive', 'release_date': datetime.datetime(2023, 3, 1, 0, 0), 'developer': 'Tero Lunkka', 'publisher': 'Tero Lunkka', 'price': '$0.20', 'description': 'third part lawnmower game series sequel succesfull lawnmower game series cut grass avoid dangerous animals', 'score': 0.7588090896606445}
{'name': 'Inclement', 'all_reviews': 'Positive', 'release_date': datetime.datetime(2023, 6, 8, 0, 0), 'developer': 'Killed Pixel Games', 'publisher': 'Killed Pixel Games', 'price': '$0.20', 'description': 'farmbuilding strategy game set h

In [10]:
user_query="Counter Strike information"
games_name=vector_search_name(user_query,collection)
print(games_name)

[{'name': 'Counter-Strike 2', 'all_reviews': 'Very Positive', 'release_date': datetime.datetime(2021, 3, 17, 0, 0), 'developer': 'Valve', 'publisher': 'Valve', 'price': '$0', 'description': 'two decades counterstrike offered elite competitive experience one shaped millions players across globe next chapter cs story begin counterstrike 2', 'score': 0.8193723559379578}]


In [ ]:
from google.genai import types

def generate_response(user_query, retrieved_games, client):
    """
    Generate a recommendation response using Gemini Flash API.

    Args:
        user_query (str): The original user query.
        retrieved_games (list): List of game metadata dicts.
        client: The initialized Google Generative AI client.

    Returns:
        str: The generated recommendation text.
    """
    context = "\n".join([
        f"{g['name']}: {g['description']}: {g['all_reviews']}: {g['release_date']}: {g['publisher']}: {g['price']}"
        for g in retrieved_games
    ])

    prompt = f"""
You are a game recommendation agent. Your task is to provide engaging and convincing recommendations to users based on their queries and the following retrieved game information.

User Query: {user_query}

Relevant Games:
{context}

Provide a detailed recommendation that:
- Highlights the most appealing aspects of the games.
- Connects the games to the user's query.
- Uses persuasive language and your own knowledge to captivate the user.
- Includes relevant information such as gameplay, reviews, release date, publisher and price.
- Make the user want to play the game.
"""

    # Use client.models.generate_content() instead of client.generate_content()
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )

    return response.text

In [ ]:
!pip install -q -U google-genai

In [ ]:
from google import genai

gemini_api_key = os.environ.get("gemini_api_key")# Replace with your Gemini API key
client = genai.Client(api_key=gemini_api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Explain how AI works in a few words"
)
print(response.text)

In [ ]:
from google.genai import types
function_declarations = [
    {
        "name": "vector_search_name",
        "description": "Search games by their title or name similarity",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "User input that includes a game title or specific name"
                }
            },
            "required": ["query"]
        }
    },
    {
        "name": "vector_search_description",
        "description": "Search games based on genre, style, or gameplay description",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "User input that includes preferences or genres"
                }
            },
            "required": ["query"]
        }
    }
]

tools = types.Tool(function_declarations=function_declarations)
config = types.GenerateContentConfig(tools=[tools])

# Send query to Gemini Flash
user_query = "Recommend shooting games in 2023 with positive reviews"
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=user_query,
    config=config,
)

# Process Gemini Flash function call and generate final response
def process_response(response, user_query, collection, client):
    candidate = response.candidates[0]

    if candidate.content.parts and candidate.content.parts[0].function_call:
        function_call = candidate.content.parts[0].function_call
        print(f"🔧 Function to call: {function_call.name}")
        print(f"📥 Arguments: {function_call.args}")

        if function_call.name == "vector_search_name":
            results = vector_search_name(function_call.args["query"], collection)
        elif function_call.name == "vector_search_description":
            results = vector_search_description(function_call.args["query"], collection)
        else:
            return "❌ Unknown function"

        # Use local model to generate the response
        return generate_response(user_query, results, client)
    else:
        print("⚠️ No function call found.")
        return candidate.content.parts[0].text

# Final response
final_response = process_response(response, user_query, collection,client)
print(final_response)